In [1]:
import torch
from torch import optim, nn
from torchvision import models, transforms
model = models.vgg16(pretrained=True)

In [2]:
class FeatureExtractor(nn.Module):
    def __init__(self, model):
        super(FeatureExtractor, self).__init__()
            # Extract VGG-16 Feature Layers
        self.features = list(model.features)
        self.features = nn.Sequential(*self.features)
            # Extract VGG-16 Average Pooling Layer
        self.pooling = model.avgpool
            # Convert the image into one-dimensional vector
        self.flatten = nn.Flatten()
            # Extract the first part of fully-connected layer from VGG16
        self.fc = model.classifier[0]

    def forward(self, x):
        # It will take the input 'x' until it returns the feature vector called 'out'
        out = self.features(x)
        out = self.pooling(out)
        out = self.flatten(out)
        out = self.fc(out) 
        return out 



In [3]:
# Initialize the model
model = models.vgg16(pretrained=True)
new_model = FeatureExtractor(model)

# Change the device to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
new_model = new_model.to(device)

In [4]:

from tqdm import tqdm
import numpy as np
from os import listdir
from os.path import isfile, join
import cv2

# Transform the image, so it becomes readable with the model
transform = transforms.Compose([
  transforms.ToPILImage(),
  transforms.CenterCrop(512),
  transforms.Resize(448),
  transforms.ToTensor()                              
])

# Will contain the feature
features = []

mypath="data/perfectly_detected_ears/train"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
# Iterate each image
for i in tqdm(onlyfiles):
    # Set the image path
    path = mypath+"/"+i
    # Read the file
    img = cv2.imread(path)
    # Transform the image
    img = transform(img)
    # Reshape the image. PyTorch model reads 4-dimensional tensor
    # [batch_size, channels, width, height]
    img = img.reshape(1, 3, 448, 448)
    img = img.to(device)
    # We only extract features, so we don't need gradient
    with torch.no_grad():
        # Extract the feature from the image
        feature = new_model(img)
    # Convert to NumPy Array, Reshape it, and save it to features variable
    features.append(feature.cpu().detach().numpy().reshape(-1))

# Convert to NumPy Array
features = np.array(features)
feature

100%|██████████| 750/750 [00:21<00:00, 34.17it/s]


tensor([[ 0.1106, -0.1079,  0.2278,  ..., -0.7626,  0.0824,  0.1163]],
       device='cuda:0')

In [5]:
from csv import reader

featureDict=dict(zip(onlyfiles,features))

featureWithIdsDict=dict()
i=0
with open('data/perfectly_detected_ears/annotations/recognition/ids.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        img,id=row
        img = img.split("/")
        if(img[0]=="train"):
            f=featureDict[img[1]]
            featureWithIdsDict[img[1]]={"features":f,"id":id}
            i+=1
featureWithIdsDict

{'0001.png': {'features': array([ 0.18542168, -0.27125838,  0.24778718, ..., -0.74696386,
         -0.09485466, -0.16550279], dtype=float32),
  'id': '100'},
 '0002.png': {'features': array([ 0.15479831, -0.01645263,  0.49826765, ..., -0.6637577 ,
          0.15456806, -0.21953224], dtype=float32),
  'id': '10'},
 '0003.png': {'features': array([ 0.23114707, -0.25365627,  0.31897116, ..., -0.61554164,
          0.04555106, -0.21321076], dtype=float32),
  'id': '11'},
 '0004.png': {'features': array([ 0.12819839, -0.11792737,  0.25846514, ..., -0.68562394,
          0.03869469,  0.12791045], dtype=float32),
  'id': '11'},
 '0005.png': {'features': array([ 0.2777681 , -0.15956497,  0.19607049, ..., -0.7429047 ,
         -0.14160839, -0.1635192 ], dtype=float32),
  'id': '11'},
 '0006.png': {'features': array([ 0.21209657, -0.13687544,  0.24283126, ..., -0.6241122 ,
          0.03550725, -0.14919531], dtype=float32),
  'id': '11'},
 '0007.png': {'features': array([ 0.2263976 , -0.3249418 

In [7]:
i=0
test_path="data/perfectly_detected_ears/test/"
res=[]
with open('data/perfectly_detected_ears/annotations/recognition/ids.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    for row in csv_reader:
        img,id=row
        img = img.split("/")
        if(img[0]=="test"):
            img = cv2.imread(test_path+img[1])
            # Transform the image
            img = transform(img)
            # Reshape the image. PyTorch model reads 4-dimensional tensor
            # [batch_size, channels, width, height]
            img = img.reshape(1, 3, 448, 448)
            img = img.to(device)
            # We only extract features, so we don't need gradient
            with torch.no_grad():
                # Extract the feature from the image
                feature = new_model(img)
            # Convert to NumPy Array, Reshape it, and save it to features variable
            f=feature.cpu().detach().numpy().reshape(-1)
            minDist=np.Infinity
            minEl=None
            intermidiateRes=dict()
            for img in featureWithIdsDict:
                dist=np.linalg.norm(f - featureWithIdsDict[img]["features"])
                if(dist<minDist):
                    minDist=dist
                    minEl=img
                if featureWithIdsDict[img]["id"] not in intermidiateRes:
                    intermidiateRes[featureWithIdsDict[img]["id"]]=dist
                else:
                    if intermidiateRes[featureWithIdsDict[img]["id"]]>dist:
                        intermidiateRes[featureWithIdsDict[img]["id"]]=dist
            newList=sorted(intermidiateRes.items(),reverse=False,key=lambda x:x[1])
            res.append((id,newList))


In [8]:
i=0
rankRes=[[] for x in range(0,100)]
for el in res:
    for r in range(0,100):
        found=False
        for i in range(r+1):
            if(el[1][i][0]==el[0]):
                found=True
        if found:
            if rankRes[r] ==[]:
                rankRes[r]=0
            rankRes[r]=rankRes[r]+1
print(rankRes)
for r in range(0,100):
    print("Res rank{} {}/{} {}%".format(r+1,rankRes[r],len(res),(rankRes[r]/len(res))*100))

IndexError: string index out of range

In [ ]:
i=0
test_path="data/perfectly_detected_ears/test/"
res=[]
from queue import PriorityQueue
from collections import Counter
with open('data/perfectly_detected_ears/annotations/recognition/ids.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    for row in csv_reader:
        img,id=row
        img = img.split("/")
        if(img[0]=="test"):
            img = cv2.imread(test_path+img[1])
            # Transform the image
            img = transform(img)
            # Reshape the image. PyTorch model reads 4-dimensional tensor
            # [batch_size, channels, width, height]
            img = img.reshape(1, 3, 448, 448)
            img = img.to(device)
            # We only extract features, so we don't need gradient
            with torch.no_grad():
                # Extract the feature from the image
                feature = new_model(img)
            # Convert to NumPy Array, Reshape it, and save it to features variable
            f=feature.cpu().detach().numpy().reshape(-1)
            minDist=np.Infinity
            minEl=None
            q = PriorityQueue()
            c = dict()
            for img in featureWithIdsDict:
                dist=np.linalg.norm(f - featureWithIdsDict[img]["features"])
                id=featureWithIdsDict[img]["id"]
                if id not in c:
                    c[id]=(dist,1)
                else:
                    c[id]=(c[id][0]+dist, c[id][1]+1)
            res.append((id,min(c.items(), key=lambda x: x[1][0]/x[1][1])[0]))

In [ ]:
i=0
for el in res:
    if(el[0]==el[1]):
        i+=1
print("Res {}/{}".format(i,len(res)))
print("Res {}%".format(i/len(res)))

Res 10/250
Res 0.04%


In [ ]:
i=0
test_path="data/perfectly_detected_ears/test/"
res=[]
from queue import PriorityQueue
from collections import Counter
with open('data/perfectly_detected_ears/annotations/recognition/ids.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    for row in csv_reader:
        img,id=row
        img = img.split("/")
        if(img[0]=="test"):
            img = cv2.imread(test_path+img[1])
            # Transform the image
            img = transform(img)
            # Reshape the image. PyTorch model reads 4-dimensional tensor
            # [batch_size, channels, width, height]
            img = img.reshape(1, 3, 448, 448)
            img = img.to(device)
            # We only extract features, so we don't need gradient
            with torch.no_grad():
                # Extract the feature from the image
                feature = new_model(img)
            # Convert to NumPy Array, Reshape it, and save it to features variable
            f=feature.cpu().detach().numpy().reshape(-1)
            minDist=np.Infinity
            minEl=None
            q = PriorityQueue()
            c = Counter()
            for img in featureWithIdsDict:
                dist=np.linalg.norm(f - featureWithIdsDict[img]["features"])
                id=featureWithIdsDict[img]["id"]
                q.put((dist,id))
            for i in range(1,100):
                first=q.get()
                c.update((first[1],))
            res.append((id,c.most_common()[0][0],c))

In [ ]:
i=0
for el in res:
    if(el[0]==el[1]):
        i+=1
print("Res {}/{}".format(i,len(res)))
print("Res {}%".format(i/len(res)))

Res 25/250
Res 0.1%
